In [ ]:
import os
import numpy as np
from slippi_db import upload_lib, preprocessing
import peppi_py

In [ ]:
prod_raw = upload_lib.get_db('prod', 'raw')
prod_slp = upload_lib.get_db('prod', 'slp')
prod_slp = list(prod_slp.find())

In [ ]:
test_slp = upload_lib.get_db('test', 'slp')
test_meta = upload_lib.get_db('test', 'meta')
test_slp = list(test_slp.find())

In [ ]:
biggest = max(prod_slp, key=lambda doc: doc['original_size'])

In [ ]:
sizes = np.array([doc['original_size'] for doc in prod_slp])

In [ ]:
np.quantile(sizes, [0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1])

In [ ]:
corrupt = [doc for doc in docs if doc['original_size'] == 0]
print(len(corrupt))

In [ ]:
prod_meta = upload_lib.get_db('prod', 'meta')
meta_docs = list(prod_meta.find())

In [ ]:
failed = [d for d in meta_docs if d.get('invalid', False)]
print(len(failed))

In [ ]:
non_parse = [f for f in failed if 'parse error' not in f['reason']]
print('non parse:', len(non_parse))
non_parse[0]

In [ ]:
import collections
reasons = collections.Counter(f['reason'] for f in failed)
reasons = sorted(reasons.items(), key=lambda kv: tuple(reversed(kv)), reverse=True)

In [ ]:
lines = [k + ', ' + str(v) + '\n' for k, v in reasons]

with open('data/reasons.txt', 'w') as f:
    f.writelines(lines)

In [ ]:
no_winner = [d for d in meta_docs if not d.get('invalid') and d['winner'] is None]
len(no_winner)

In [ ]:
from slippi_db import test_peppi
from importlib import reload
reload(test_peppi)

In [ ]:
reload(test_peppi)
singles_metas = test_peppi.get_singles_info('prod')

In [ ]:
len(singles_metas)

In [ ]:
total_frames = sum(124 + m['lastFrame'] for m in singles_metas)
total_frames / (60 * 60 * 60)

In [ ]:
slp_to_raw = {d['key']: d['raw_key'] for d in test_slp}

In [ ]:
keys = [d['key'] for d in non_parse]
raw_keys = set([slp_to_raw[k] for k in keys])
print(raw_keys)

In [ ]:
dir = 'data/test/start'
for key in keys[:1]:
    upload_lib.download_slp_locally('test', key, 'data/test/start/')

In [ ]:
games = []
path = 'data/test/start/'
for name in os.listdir(path):
    games.append(peppi_py.game(os.path.join(path, name)))

In [ ]:
def analyze_game(game):
    start_ports = [p['port'] for p in game['start']['players']]
    frame_ports = sorted(game['frames'][0]['ports'])
    print(start_ports, frame_ports)

In [ ]:
path = 'data/test/18_29_53 Kirby + Falco.slp'
game = peppi_py.game(path)

In [ ]:
last_frame = game['frames'][-1]
stocks = [player['leader']['post']['stocks'] for player in last_frame['ports'].values()]
stocks

In [ ]:
winner = None
last_frame = game['frames'][-1]
stock_counts = {}
for index, player in last_frame['ports'].items():
    stock_counts[int(index)] = player['leader']['post']['stocks'].as_py()
losers = [p for p, s in stock_counts.items() if s == 0]
if losers:
    winners = [p for p, s in stock_counts.items() if s > 0]
    if len(winners) == 1:
        winner = winners[0]
print(winner)

In [ ]:
x = stock_counts[0]

In [ ]:
x.as_py()

In [ ]:
%pdb

In [ ]:
local_games = []
path = os.path.expanduser('~/Slippi')
for name in os.listdir(path):
    try:
        game = peppi_py.game(os.path.join(path, name))
    except OSError:
        continue
    local_games.append(game)

    if len(local_games) == 20:
        break

In [ ]:
len(local_games)

In [ ]:
for g in local_games:
    analyze_game(g)

In [ ]:
path = 'data/test/start/18_29_53 Kirby + Falco.slp'
preprocessing.assert_same_parse(path)

In [ ]:
%pdb